# Main.py

In [2]:
#import asyncio
# # ---------- database ------------
import pandas as pd
import sqlite3 as db
# # ---------- alpar libs ----------
import config 
from data_ops import fetch, latest_workday, scrape, utils, update_daily_quotes

from pathlib import Path
import requests 
import more_itertools as mit
import itertools as it

cfg = config.configs 

latest_workday

20230219

In [3]:
conn = db.connect('./db/catalogue.db')

In [2]:
# -------------extras: settings, sample df------------
# ---------------------settings------------
# 
# with pd.option_context('display.max_colwidth', None,
#                        'display.max_columns', None,
#                        'display.max_rows', None):
#     display(types_df)
#
# -------------sample dummy dataframe-------------------
# 
# prev = pd.DataFrame({
#     'name':['first_name','first_name', 'first_name', 'first_name', 'first_name'],
#     'fetch_date': ['20230122', '20230123', '20230122', '20230123', '20230123'],
#     'id': [100,100,100,125,126]}, 
#     index=['917857106093847', '1050751214677134', '2589887561569709', '3542690854557886', '3772339654185462'])
# update = pd.DataFrame({
#     'name':['second_old_name', 'first_new_name', 'third_old_name_updated', 'second_new_name'],
#     'fetch_date': ['20230123', '20230121', '20230122', '20230123'],
#     'id': [200,200,130,130]}, 
#     index=['1050751214677134', '3542690854456286', '2589887561569709', '3772339112185462'])

# headers = {
#                     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0',
#                     'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
#                     'Accept-Language': 'en-US,en;q=0.5',
#                     # 'Accept-Encoding': 'gzip, deflate',
#                     'Connection': 'keep-alive',
#                     # 'Cookie': '_ga=GA1.2.1125034676.1645178849',
#                     'Upgrade-Insecure-Requests': '1',
#                     'Pragma': 'no-cache',
#                     'Cache-Control': 'no-cache',
#                 }

# import numpy as np
# dates = pd.date_range('2023-02-01', periods=8)
# df = pd.DataFrame(np.random.randn(8,4), columns=['A','B','C','D'], index=dates)
# df2 = pd.DataFrame(np.random.randn(8,4), columns=['A','B','C','D'], index=dates)

In [3]:
all_insts = fetch.instruments(0)
# احتمالا اونایی رو میده ک تا حالا معامله داشته باشن
# اونایی ک تا این تاریخ معامله نداشته باشن برنمیگردونه

In [4]:
query = 'select id from instrument where tableu_code != "6"'
# query = 'select instrument.id, instrument.ticker, instrument.tableu_code, identity.market from instrument left join identity on instrument.isin = identity.isin'
insts = pd.read_sql(query, con=conn, index_col='id')

In [7]:
insts[insts.index == '41934470778361119']

,ticker,tableu_code,market
id,,,
41934470778361119,45-پیمانکاری6,6,None


In [7]:
await scrape.loop_coroutines_daily_quotes(insts.index[0:40])

100%|██████████| 40/40 [00:00<00:00, 51.90it/s]


,instrument_id,date,close,last,transaction_count,volume,value_traded,low,high,dday,open
0,1123534346391630,20081206,1244.20,1244.20,3,20020,20006740.00,1244.20,1244.20,0.00,1244.20
1,1123534346391630,20081207,1244.20,1244.20,0,0,0.00,1244.20,1244.20,1244.20,0.00
2,1123534346391630,20081208,1244.20,1244.20,1,10000,10000000.00,1244.20,1244.20,1244.20,1244.20
3,1123534346391630,20081210,1244.20,1244.20,0,0,0.00,1244.20,1244.20,1244.20,0.00
4,1123534346391630,20081213,1244.20,1244.20,11,93800,93800000.00,1244.20,1244.20,1244.20,1244.20
...,...,...,...,...,...,...,...,...,...,...,...
3416,1123534346391630,20230201,200547.00,200547.00,9233,351666791,623317249434.00,199564.00,201489.00,119851702.22,201489.00
3417,1123534346391630,20230205,194566.00,194566.00,4839,190078495,386070538839.00,194566.00,199455.00,116277595.90,199455.00
3418,1123534346391630,20230206,187650.00,187650.00,8142,483872070,752568189919.00,187650.00,192189.00,112144072.68,192189.00
3419,1123534346391630,20230207,191204.00,191204.00,3128,144316597,276612786872.00,189937.00,191399.00,114268148.98,189937.00


### Scraping Trade History

### Instrument Categorization

In [6]:
# instrument.id, instrument.isin, instrument.ticker, instrument.description, instrument.market_code, instrument.industry_subsector_code, instrument_type.code, instrument_type.category, instrument_type.sub_category from instrument
inst_type = pd.read_sql('select * from instrument join instrument_type join identity on instrument.type_code = instrument_type.code and instrument.isin = identity.isin', con=conn, index_col='id')
# types = pd.read_sql('select * from instrument_type', con=conn)
# instruments = pd.read_sql('select * from instrument', con=conn, index_col='id')

inst_type.drop_duplicates('isin', inplace=True)

with pd.option_context(
    'display.max_colwidth', None,
    'display.max_columns', 34,
    'display.max_rows', 4):
    display(inst_type)

In [ ]:
insts[insts['tableu_code'] == '1']['market'].unique()

array(['بازار اول فرابورس', 'بازار اول (تابلوی اصلی) بورس', '-'],
      dtype=object)

In [ ]:
insts['tableu_code'].unique()

array(['8', '5', '4', '1', '7', '3', '9', '6'], dtype=object)

In [ ]:
insts[insts['tableu_code'] == '8']

,ticker,tableu_code,market
id,,,
4054414296721,ضلبخند202,8,-
133407505805365,طبدر2033,8,-
183419485240889,ضفگستر1008,8,-
1629946718091595,طسلا7005,8,-
1665565213994526,ضفگستر1010,8,-
...,...,...,...
69843691250564003,ضلبخند508,8,-
70304393734444484,طسلا7001,8,-
70950258269733388,طخود0107,8,-


### Get Trade History (Ticks/Trade Details)

In [3]:
response = req.get(
    f'http://cdn.tsetmc.com/api/Trade/GetTradeHistory/{inst_id}/{desired_date}/false', verify=False, headers=headers)
json_data = json.loads(response.text)
ticks = pd.DataFrame(json_data['tradeHistory']).set_index('nTran').sort_index()
ticks


,insCode,dEven,hEven,qTitTran,pTran,qTitNgJ,iSensVarP,pPhSeaCotJ,pPbSeaCotJ,iAnuTran,xqVarPJDrPRf,canceled
nTran,,,,,,,,,,,,
1,None,0,90037,300000,2941.0,0, ,0.0,0.0,0,0.0,0
2,None,0,90037,100000,2941.0,0, ,0.0,0.0,0,0.0,0
3,None,0,90037,43368,2941.0,0, ,0.0,0.0,0,0.0,0
4,None,0,90037,11960,2941.0,0, ,0.0,0.0,0,0.0,0
5,None,0,90037,10218,2941.0,0, ,0.0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6711,None,0,122958,20000,3067.0,0, ,0.0,0.0,0,0.0,0
6712,None,0,122959,20000,3067.0,0, ,0.0,0.0,0,0.0,0
6713,None,0,122959,14250,3067.0,0, ,0.0,0.0,0,0.0,0


In [5]:
ticks['insCode'] = inst_id
ticks['dEven'] = pd.to_datetime(desired_date,format='%Y%m%d')
#ticks['ts'] = pd.to_datetime(ticks['hEven'], format='%H%M%S')
ticks['timestamp']= ticks['dEven'].astype(str) + pd.to_datetime(ticks['hEven'], format='%H%M%S').astype(str)

In [6]:
ticks

,insCode,dEven,hEven,qTitTran,pTran,qTitNgJ,iSensVarP,pPhSeaCotJ,pPbSeaCotJ,iAnuTran,xqVarPJDrPRf,canceled,timestamp
nTran,,,,,,,,,,,,,
1,43362635835198978,2022-08-21,90037,300000,2941.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 09:00:37
2,43362635835198978,2022-08-21,90037,100000,2941.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 09:00:37
3,43362635835198978,2022-08-21,90037,43368,2941.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 09:00:37
4,43362635835198978,2022-08-21,90037,11960,2941.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 09:00:37
5,43362635835198978,2022-08-21,90037,10218,2941.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 09:00:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6711,43362635835198978,2022-08-21,122958,20000,3067.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 12:29:58
6712,43362635835198978,2022-08-21,122959,20000,3067.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 12:29:59
6713,43362635835198978,2022-08-21,122959,14250,3067.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 12:29:59


In [ ]:
ticks.resample()

### Filtering & Ploting

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'


In [15]:
# Today's MarketWatch! DataFrame
traded_today_df = pd.DataFrame([data[instrument].iloc[-1] for instrument in instruments_traded_so_far if (data[instrument].index[-1] == last_workday)])


In [6]:
high_value = op.filter_by_value(instrument_tickers=instruments_traded_today, database=data, factor=2, make_df=True)
high_value


,ent code,name,namad,nam,j-date,open,high,low,last,close,...,volume MA 5,volume MA 9,volume MA 3,volume MA 26,volume MA 52,value MA 3,value MA 5,value MA 9,value MA 26,value MA 52
2022-09-10,TB77,TreasuryBill230109-a,اخزا902,اسنادخزانه-م2بودجه99-011019-ت,14010619,937000.0,937000.0,935220.0,936500.0,936910.0,...,28700.0,2.518889e+04,4.353333e+04,2.692308e+04,4.750385e+04,4.077619e+10,2.687331e+10,2.352258e+10,2.494468e+10,4.336457e+10
2022-09-10,TB83,TreasuryBill230828-a,اخزا908,اسنادخزانه-م8بودجه99-020606-ت,14010619,828870.0,829430.0,828000.0,829430.0,829330.0,...,28160.0,2.363333e+04,3.763333e+04,1.716538e+04,4.509423e+04,3.120696e+10,2.332243e+10,1.953626e+10,1.408507e+10,3.614158e+10
2022-09-10,TB91,TreasuryBill230819-a,اخزا916,اسنادخزانه-م16بودجه99-020528-ت,14010619,842180.0,842180.0,842180.0,842180.0,842180.0,...,12409.8,9.119333e+03,1.833533e+04,6.203263e+03,6.203263e+03,1.541292e+10,1.035659e+10,7.508267e+09,4.911234e+09,4.911234e+09
2022-09-10,SALF,AminTadbirFardaETF-a,الماس,صندوق_س.امين_تدبيرگران_فردا-س-ت,14010619,111000.0,114000.0,111000.0,111000.0,112097.0,...,13507.4,1.236522e+04,2.065900e+04,1.207558e+04,1.582867e+04,2.315368e+09,1.513163e+09,1.393643e+09,1.330181e+09,1.777295e+09
2022-09-10,TOSZ,TOSAN-a,توسن,توسعه_سامانه_ي_نرم_افزاري_نگين-ت,14010619,14990.0,15520.0,14550.0,15520.0,15320.0,...,2606217.0,1.957540e+06,3.788956e+06,1.314413e+06,1.201066e+06,5.849467e+10,4.178938e+10,3.372620e+10,2.487222e+10,2.360080e+10
2022-09-10,KSHJ,S*IRI Marine Co.-a,حکشتی,كشتيراني_جمهوري_اسلامي_ايران-ت,14010619,23510.0,23510.0,22560.0,22570.0,22570.0,...,14556392.4,1.709113e+07,1.924198e+07,1.659934e+07,2.091673e+07,4.410619e+11,3.382885e+11,4.153053e+11,3.963030e+11,4.603504e+11
2022-09-10,KKHP,S*Kerman Khodro E.-a,خکرمان,گروه_اقتصادي_كرمان_خودرو-ت,14010619,3648.0,3648.0,3583.0,3648.0,3647.0,...,14766569.0,1.165854e+07,2.159768e+07,1.558829e+07,1.438011e+07,7.805133e+10,5.289414e+10,4.149542e+10,5.428829e+10,4.874815e+10
2022-09-10,INFO,Inf. Services-a,رانفور,خدمات‌انفورماتيك‌-ت,14010619,3930.0,4049.0,3840.0,4049.0,4023.0,...,2740031.4,2.704766e+06,3.020928e+06,3.143687e+06,3.256747e+06,1.205973e+10,1.096140e+10,1.088318e+10,1.251370e+10,1.277916e+10
2022-09-10,ZAKF,Zarin Korosh ETF-a,زرین,صندوق_س._سهام_زرين_كوروش-س-ت,14010619,12000.0,12288.0,11930.0,11950.0,12086.0,...,117864.0,9.022900e+04,1.429957e+05,1.223091e+05,1.599467e+05,1.728267e+09,1.429458e+09,1.098126e+09,1.509757e+09,2.021369e+09
2022-09-10,BTEJ,BTEJ-O-14010720-a,طجار7047,اختيارف_وتجارت-1700-1401/07/20-ت,14010619,200.0,200.0,150.0,175.0,159.0,...,245.0,2.534444e+02,3.613333e+02,2.326500e+02,2.326500e+02,5.696067e+07,3.640440e+07,2.872356e+07,2.106390e+07,2.106390e+07


In [13]:
high_value_plot = px.bar(high_value, 'namad', [
    'value traded', 'value MA 3', 'value MA 5', 'value MA 9', 'value MA 26'], barmode='group')

viz.beautify_chart(high_value_plot)
high_value_plot.show(config=viz.config)


### Fundamentals (Amin)

In [2]:
amin= pd.read_excel('D:/Alpar/assisstant/dash/inputs/amin.xlsx')
amin.set_index('نماد',inplace=True)

In [3]:
fund= pd.DataFrame()
fund['eps ttm'] =amin['قیمت پایانی ریال'] / amin['P/E TTM']
fund['eps f growth'] =amin['قیمت پایانی ریال'] / amin['P/E کارشناسی رشدی']
fund['fiscal year ends on'] =amin['سال مالی منتهی به']
fund['fiscal month'] = amin['آخرین ماه']
fund['activity type'] = amin['نوع فعالیت']
fund['firm'] = amin['شرکت']

In [4]:
fund

,eps ttm,eps f growth,fiscal year ends on,fiscal month,activity type,firm
نماد,,,,,,
فولاد,2411.474666,3779.639390,1401/12/29,M03,تولیدی,فولاد مبارکه اصفهان
فاسمین,3898.692333,8502.915714,1401/12/29,M03,تولیدی,کالسیمین
خراسان,18132.129959,42230.530878,1401/12/29,M03,تولیدی,پتروشیمی خراسان
شپدیس,28070.350333,43444.511091,1401/06/31,M09,تولیدی,پتروشیمی پردیس
شغدیر,15189.028462,22804.475874,1401/12/29,M03,تولیدی,پتروشیمی غدیر
...,...,...,...,...,...,...
مادیرا,138.786268,NaN,1401/12/29,M03,تولیدی,صنایع ماشین های اداری ایران
لپارس,1382.043333,NaN,1401/12/29,M03,تولیدی,کارخانجات پارس الکتریک
کبافق,1247.232099,NaN,1401/12/29,M03,تولیدی,معادن بافق
